In [1]:
import sys
sys.path.append('../')

import spacy
from dataset import collate_batch_amazon, get_data_loader_amazon_bert
from functools import partial

from utils import model_and_tokenizer_from_spec_cls, process_df, init_random_seed
from metrics import *

from tqdm import tqdm
import pandas as pd
import numpy as np
import random
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch

In [2]:
init_random_seed(9001)

# Modify following values according to desired flow

In [3]:
original_model_path = "bert-base-uncased"
model_dir = "../models/"
model_fname = ''
test_set_fname = ''
mode = 'random' # Can be 'random', 'complete' or 'subsets'

In [4]:
spacy_tokenizer = spacy.blank('en')
def reduce_title(title,max_len,tokenizer):
    tokens = tokenizer(title)
    return tokens[:max_len].text

In [5]:
test = pd.read_csv(test_set_fname, keep_default_na=False, na_values=['$$$__$$$'])
test['title'] = test['title'].apply(lambda x: " ".join(x.split()))
test = process_df(test, spacy_tokenizer) if mode in ['random', 'subsets'] else test
if mode == 'complete':
    test['tok_len'] = test['title'].apply(lambda x: len(spacy_tokenizer(x)))

In [6]:
original_token_lengths = [12,]
prefix_lengths = list(range(1,7))


stats = {}
model, tokenizer = model_and_tokenizer_from_spec_cls(original_model_path,num_classes=test['label'].nunique())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = os.path.join(model_dir,model_fname)
model.load_state_dict(torch.load(model_path))
model = model.to(device)
model.eval()

for length in original_token_lengths:
    print("working on length",length)
    stats[length] = {}
    for prefix_length in prefix_lengths:
        stats[length][prefix_length] ={}
        tmp = test.copy(deep=True)
        tmp = tmp[tmp.tok_len==length].reset_index(drop=True)
        tmp['title'] = tmp['title'].apply(lambda x: reduce_title(x,prefix_length,spacy_tokenizer))
        collate_fn = partial(collate_batch_amazon, pad_token_id=tokenizer.pad_token_id)
        test_dl = get_data_loader_amazon_bert(tmp,tokenizer, collate_fn, batch_size=64, num_workers=4)

        all_labels = []
        all_preds_scores = []
        with torch.no_grad():
            for step,batch in enumerate(tqdm(test_dl)):
                inp, labels = batch
                model_input = tuple([item.to(device) for item in inp])
                labels = labels.to(device)
                outputs = model(model_input)
                all_preds_scores.extend(list(outputs.cpu().numpy()))
                all_labels.extend(list(labels.cpu().numpy()))

        all_preds_scores, all_labels = np.array(all_preds_scores),np.array(all_labels)
        all_preds = np.argmax(all_preds_scores,axis=1)
        
        precision_stats = get_all_classes_stats(precision,all_preds,all_labels)
        macro_precision = np.mean([precision_stats[i] for i in precision_stats])*100
        recall_stats = get_all_classes_stats(recall,all_preds,all_labels)
        macro_recall = np.mean([recall_stats[i] for i in recall_stats])*100
        accuracy_value = accuracy(all_preds,all_labels)*100
        
        stats[length][prefix_length]['MACRO_PRECISION'] = macro_precision
        stats[length][prefix_length]['MACRO_RECALL'] = macro_recall
        stats[length][prefix_length]['ACCURACY'] = accuracy_value
        
        for k in [3,5]:
            hits, hit_rate = rank_predictions_metrics(all_preds_scores, all_labels, k)
            stats[length][prefix_length][f'HITS@{k}'] = hit_rate

loading pretrained model_utils...


/Users/ggoren/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exa

loading tokenizer...
Creating fine-tuning architecture for multiclass classification (Num Classes: 2)
working on length 12


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:14<00:00,  7.11s/it]
